In [1]:
import requests
import base64

In [2]:
CLIENT_ID='a7843066ef5440a483d97153e51725ad'
CLIENT_SECRET='ff4a6b5ea4c24f11a0548232f9a5dd5a'

client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}

response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained successfully.")
else:
    print("Error obtaining access token.")



Access token obtained successfully.


In [3]:
 !pip install spotipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 6.4 MB/s eta 0:00:00


In [4]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

In [5]:
#Get the playlist based on spotify playlist id

def get_playlist_from_spotify(playlistid, token):
  sp=spotipy.Spotify(auth=token)
  alltracks=sp.playlist_tracks(playlistid,fields='items(track(id,name,artists,album(id,name)))')
  data=[]
  for trackinfo in alltracks['items']:
    track=trackinfo['track']
    name=track['name']
    artists = ', '.join([artist['name'] for artist in track['artists']])
    album_name = track['album']['name']
    album_id = track['album']['id']
    track_id = track['id']
    audio_features=sp.audio_features(track_id)[0] if track_id is not None else None
    thistrack=sp.track(track_id) if track_id is not None else None
    album_info = sp.album(album_id) if album_id is not None else None
    release_date = album_info['release_date'] if album_info else None
    popularity=thistrack['popularity'] if thistrack else None
    track_details={
        'Name':name,
        'Artists':artists,
        'Album Name':album_name,
        'Track ID':track_id,
        'Release Date':release_date,
        'Popularity':popularity,
        'Danceability':audio_features['danceability'] if audio_features else None,
        'Energy':audio_features['energy'] if audio_features else None,
        'Key':audio_features['key'] if audio_features else None,
        'Loudness':audio_features['loudness'] if audio_features else None,
        'Mode':audio_features['mode'] if audio_features else None,
        'Speechiness':audio_features['speechiness'] if audio_features else None,
        'Acousticness':audio_features['acousticness'] if audio_features else None,
        'Instrumentalness':audio_features['instrumentalness'] if audio_features else None,
        'Liveness':audio_features['liveness'] if audio_features else None,
        'Valence':audio_features['valence'] if audio_features else None,
        'Tempo':audio_features['tempo'] if audio_features else None,

    }
    data.append(track_details)

  df=pd.DataFrame(data)
  return df



In [6]:
music_df = get_playlist_from_spotify('3IJB33bPY9n37Wy1CcPrlj', access_token)

In [7]:
print(music_df['Name'])

0                                       Patola
1     Why This Kolaveri Di? - The Soup of Love
2                               Karuppinazhaku
3                             Dil Diyan Gallan
4             Ee Ulakin (From "King of Kotha")
                        ...                   
95                 Falak Tak (slowed & reverb)
96                 London View - Bootleg Remix
97                                     ilomilo
98               Mask Off (Politik Trap Remix)
99           Runaway (TikTok Classics Version)
Name: Name, Length: 100, dtype: object


In [8]:
print(music_df.isnull().sum())

Name                0
Artists             0
Album Name          0
Track ID            8
Release Date        8
Popularity          8
Danceability        8
Energy              8
Key                 8
Loudness            8
Mode                8
Speechiness         8
Acousticness        8
Instrumentalness    8
Liveness            8
Valence             8
Tempo               8
dtype: int64


In [14]:
music_df=music_df.dropna()

In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
scaler=MinMaxScaler()
song_features= music_df[['Danceability', 'Energy', 'Key', 'Loudness', 'Mode', 'Speechiness', 'Acousticness','Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values
song_features_scaled=scaler.fit_transform(song_features)
song_features

array([[5.79000e-01, 8.45000e-01, 3.00000e+00, ..., 9.39000e-02,
        5.30000e-01, 1.75657e+02],
       [8.20000e-01, 7.85000e-01, 5.00000e+00, ..., 3.72000e-01,
        8.94000e-01, 1.40013e+02],
       [6.09000e-01, 8.64000e-01, 4.00000e+00, ..., 8.41000e-02,
        7.76000e-01, 1.84056e+02],
       ...,
       [4.53000e-01, 5.31000e-01, 1.00000e+00, ..., 1.50000e-01,
        3.24000e-01, 9.17180e+01],
       [8.28000e-01, 6.87000e-01, 5.00000e+00, ..., 1.24000e-01,
        2.08000e-01, 1.19006e+02],
       [3.65000e-01, 4.10000e-01, 1.10000e+01, ..., 1.06000e-01,
        1.25000e-01, 1.71641e+02]])

In [23]:
def recommendations_fn(input_song_name, num_recommendations=10):
    if input_song_name not in music_df['Name'].values:
        print(f"'{input_song_name}' not found.")
        return

    input_song_index = music_df[music_df['Name'] == input_song_name].index[0]

    similarity_scores = cosine_similarity([song_features_scaled[input_song_index]], song_features_scaled)

    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]

    content_based_recommendations = music_df.iloc[similar_song_indices][['Name', 'Artists', 'Album Name','Popularity']]

    return content_based_recommendations



In [24]:
input_song_name = "Guli Mata"
recommendations = recommendations_fn(input_song_name, num_recommendations=5)
print(f"recommended songs for '{input_song_name}':")
print(recommendations)

recommended songs for 'Guli Mata':
                     Name                                     Artists  \
46  Deewane Hum Nahi Hote                                Aditya Yadav   
20            Paani Paani                        Badshah, Aastha Gill   
32            Hridhayavum  Vineeth Sreenivasan, Jyotsna Radhakrishnan   
3        Dil Diyan Gallan    Vishal-Shekhar, Atif Aslam, Irshad Kamil   
50  Yaen Ennai Pirindhaai                                  Sid Sriram   

                                        Album Name  Popularity  
46                           Deewane Hum Nahi Hote        58.0  
20                                     Paani Paani        53.0  
32  Note Book (Original Motion Picture Soundtrack)        55.0  
3                                  Tiger Zinda Hai        71.0  
50                                   Adithya Varma        58.0  
